In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import time

In [2]:
import ssl  # магия чтобы fetch_openml сработал
ssl._create_default_https_context = ssl._create_unverified_context

# <p style="color: #6c89c6;">Работу выполнил: Таратин Артём ПМ22-1<br>Вариант: 13</p>

In [3]:
modeles_comp = pd.DataFrame({'Модель': [], 'Время': [], 'Точнось': []})
modeles_comp

,Модель,Время,Точнось


## 1. (1 балл) Загрузите предложенный вам датасет с помощью функции `sklearn.datasets.fetch_openml`. Выведите текстовое описание загруженного датасета. Обозначьте целевую переменную за `y`, а остальные данные за `X`.

In [4]:
dataset = fetch_openml('diggle_table_a2', parser='auto', version=1)
print(dataset.DESCR)

**Author**:   
**Source**: Unknown - Date unknown  
**Please cite**:   

DATA-SETS FROM DIGGLE, P.J. (1990). TIME SERIES : A BIOSTATISTICAL
INTRODUCTION. Oxford University Press.

Table: Table A2 Wool prices


Information about the dataset
CLASSTYPE: numeric
CLASSINDEX: none specific

Downloaded from openml.org.


In [5]:
dataset_df = pd.DataFrame(dataset.frame)
dataset_df.columns = ['y', *dataset_df.columns[1:]]
dataset_df

,y,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9
0,76,33,33,262.0,307.0,1.1718,5.5683,5.7268,0.1585
1,76,34,34,259.0,306.0,1.1815,5.5568,5.7236,0.1668
2,76,35,35,257.0,306.0,1.1907,5.5491,5.7236,0.1745
3,76,36,36,262.0,306.0,1.1679,5.5683,5.7236,0.1553
4,76,37,37,264.0,306.0,1.1591,5.5759,5.7236,0.1477
...,...,...,...,...,...,...,...,...,...
305,84,20,436,495.0,786.0,1.5879,6.2046,6.6670,0.4624
306,84,21,437,495.0,786.0,1.5879,6.2046,6.6670,0.4624
307,84,22,438,497.0,786.0,1.5815,6.2086,6.6670,0.4584
308,84,25,441,496.0,786.0,1.5847,6.2066,6.6670,0.4604


In [6]:
X = dataset_df.iloc[:, 1:]
y = dataset_df.iloc[:, 0]
X.shape, y.shape

((310, 8), (310,))

## 2. (1 балл) Выведите основную статистическую информацию о данных. Сделайте количественное описание датасета: число строк (объектов), число столбцов (признаков), статистику по признакам, количество классов (значений целевой переменной).

In [7]:
dataset_df.shape, X.shape, y.shape

((310, 9), (310, 8), (310,))

In [8]:
dataset_df.describe()

,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9
count,310.000000,310.000000,310.000000,310.000000,310.000000,310.000000,310.00000,310.000000
mean,26.587097,234.922581,387.183871,496.103226,1.265265,5.943422,6.17463,0.231207
std,15.335635,120.804559,66.762306,124.330267,0.116450,0.178233,0.25636,0.089691
min,2.000000,33.000000,257.000000,306.000000,1.077800,5.549100,5.72360,0.075000
25%,12.000000,127.250000,316.000000,357.000000,1.155950,5.755700,5.87770,0.144925
50%,25.000000,233.500000,405.000000,524.000000,1.272000,6.003900,6.26150,0.240550
75%,41.000000,338.750000,434.500000,576.000000,1.323075,6.074150,6.35610,0.280025
max,51.000000,442.000000,499.000000,791.000000,1.604500,6.212600,6.67330,0.472800


In [9]:
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   y       310 non-null    category
 1   col_2   310 non-null    int64   
 2   col_3   310 non-null    int64   
 3   col_4   310 non-null    float64 
 4   col_5   310 non-null    float64 
 5   col_6   310 non-null    float64 
 6   col_7   310 non-null    float64 
 7   col_8   310 non-null    float64 
 8   col_9   310 non-null    float64 
dtypes: category(1), float64(6), int64(2)
memory usage: 20.2 KB


In [10]:
uq = y.unique()
print(f'Классы: {uq}\nКоличество: {len(uq)}')

Классы: ['76', '77', '78', '79', '80', '81', '82', '83', '84']
Categories (9, object): ['76', '77', '78', '79', ..., '81', '82', '83', '84']
Количество: 9


In [11]:
dataset_df.dtypes

y        category
col_2       int64
col_3       int64
col_4     float64
col_5     float64
col_6     float64
col_7     float64
col_8     float64
col_9     float64
dtype: object

## 3. (1 балл) Убедитесь, что данные пригодны для моделирования. В данных не должно быть пропущенных значений, ве признаки должны быть численными. Если эти условия нарушаются, исправьте это.

In [12]:
dataset_df.isna().any()

y        False
col_2    False
col_3    False
col_4    False
col_5    False
col_6    False
col_7    False
col_8    False
col_9    False
dtype: bool

In [13]:
# dataset_df['y'] = dataset_df['y'].astype(np.int64)
# dataset_df.dtypes

In [14]:
# X, y = dataset_df.iloc[:, 1:], dataset_df.iloc[:, 0]
# X.dtypes, y.dtypes

## 4. (2 балла) Обучите модель логистической регрессии на рассматриваемых данных из библиотеки `sklearn`. Рассчитайте метрики accuracy и выведите таблицу классификации.

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=1)

In [16]:
model_log = LogisticRegression(max_iter=1500)
tm1 = time.time()
model_log.fit(X_train, y_train)
tm = time.time() - tm1
y_pred = model_log.predict(X_test)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
ac = metrics.accuracy_score(y_test, y_pred)
modeles_comp.loc[modeles_comp.shape[0]] = ['linear', tm, ac]
ac

1.0

In [18]:
print(confusion_matrix(y_test, y_pred))

[[ 9  0  0  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  0  0]
 [ 0  0 14  0  0  0  0  0  0]
 [ 0  0  0 14  0  0  0  0  0]
 [ 0  0  0  0 12  0  0  0  0]
 [ 0  0  0  0  0 16  0  0  0]
 [ 0  0  0  0  0  0 14  0  0]
 [ 0  0  0  0  0  0  0 10  0]
 [ 0  0  0  0  0  0  0  0 10]]


In [19]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          76       1.00      1.00      1.00         9
          77       1.00      1.00      1.00        10
          78       1.00      1.00      1.00        14
          79       1.00      1.00      1.00        14
          80       1.00      1.00      1.00        12
          81       1.00      1.00      1.00        16
          82       1.00      1.00      1.00        14
          83       1.00      1.00      1.00        10
          84       1.00      1.00      1.00        10

    accuracy                           1.00       109
   macro avg       1.00      1.00      1.00       109
weighted avg       1.00      1.00      1.00       109



## 5. (2 балла) Обучите полиномиальную модель классификации. Рассчитайте метрики accuracy и выведите таблицу классификации. Попробуйте разные степени полинома и выберите ту, которая работает лучше.

In [20]:
degrees = list(range(2, 6))
for degree in degrees:
    poly = PolynomialFeatures(degree=degree)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)

    model_log = LogisticRegression(max_iter=25000)
    
    tm1 = time.time()
    model_log.fit(X_train_poly, y_train)
    tm = time.time() - tm1
    
    y_pred = model_log.predict(X_test_poly)
    
    accuracy_score = metrics.accuracy_score(y_test, y_pred)
    
    modeles_comp.loc[modeles_comp.shape[0]] = [f'poly {degree}', tm, accuracy_score]
    
    print(f'Степень: {degree} | accuracy_score: {accuracy_score:.4f} | confusion_matrix:\n')
    print(confusion_matrix(y_test, y_pred), end='\n'*2+'-'*75+'\n')

Степень: 2 | accuracy_score: 1.0000 | confusion_matrix:

[[ 9  0  0  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  0  0]
 [ 0  0 14  0  0  0  0  0  0]
 [ 0  0  0 14  0  0  0  0  0]
 [ 0  0  0  0 12  0  0  0  0]
 [ 0  0  0  0  0 16  0  0  0]
 [ 0  0  0  0  0  0 14  0  0]
 [ 0  0  0  0  0  0  0 10  0]
 [ 0  0  0  0  0  0  0  0 10]]

---------------------------------------------------------------------------
Степень: 3 | accuracy_score: 0.9725 | confusion_matrix:

[[ 9  0  0  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  0  0]
 [ 0  0 14  0  0  0  0  0  0]
 [ 0  0  0 14  0  0  0  0  0]
 [ 0  0  0  0 11  1  0  0  0]
 [ 0  0  0  0  0 16  0  0  0]
 [ 0  0  0  0  0  0 14  0  0]
 [ 0  0  0  0  0  0  0  8  2]
 [ 0  0  0  0  0  0  0  0 10]]

---------------------------------------------------------------------------
Степень: 4 | accuracy_score: 0.9908 | confusion_matrix:

[[ 9  0  0  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  0  0]
 [ 0  0 14  0  0  0  0  0  0]
 [ 0  0  0 14  0  0  0  0  0]
 [ 0  0  0  0

#### 2 степень оказалась лучше по аккуратности в данном тесте

### 6. (2 балла) Обучите модель классификации по методу опорных векторов. Рассчитайте метрики accuracy и выведите таблицу классификации. Попробуйте разные ядерные функции и выберите ту, которая работает лучше.

In [21]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
for kernel in kernels:
    model_svm = SVC(kernel=kernel)
    
    tm1 = time.time()
    model_svm.fit(X_train, y_train)
    tm = time.time() - tm1
    
    y_pred = model_svm.predict(X_test)
    
    accuracy = metrics.accuracy_score(y_test, y_pred)
    
    modeles_comp.loc[modeles_comp.shape[0]] = [f'svc {kernel}', tm, accuracy]
    
    print(f'Ядро: {kernel}, accuracy_score: {accuracy:.4f}\n')
    print(confusion_matrix(y_test, y_pred), end='\n'*2+'-'*75+'\n')

Ядро: linear, accuracy_score: 1.0000

[[ 9  0  0  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  0  0]
 [ 0  0 14  0  0  0  0  0  0]
 [ 0  0  0 14  0  0  0  0  0]
 [ 0  0  0  0 12  0  0  0  0]
 [ 0  0  0  0  0 16  0  0  0]
 [ 0  0  0  0  0  0 14  0  0]
 [ 0  0  0  0  0  0  0 10  0]
 [ 0  0  0  0  0  0  0  0 10]]

---------------------------------------------------------------------------
Ядро: poly, accuracy_score: 0.7064

[[ 0  9  0  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  0  0]
 [ 0 14  0  0  0  0  0  0  0]
 [ 0  2  0 12  0  0  0  0  0]
 [ 0  0  0  2 10  0  0  0  0]
 [ 0  0  0  0  0 16  0  0  0]
 [ 0  0  0  0  0  2 12  0  0]
 [ 0  0  0  0  0  0  2  8  0]
 [ 0  0  0  0  0  0  0  1  9]]

---------------------------------------------------------------------------
Ядро: rbf, accuracy_score: 0.5688

[[ 0  9  0  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  0  0]
 [ 0 14  0  0  0  0  0  0  0]
 [ 0  3  0  9  2  0  0  0  0]
 [ 0  0  0  0 12  0  0  0  0]
 [ 0  0  0  0 10  0  6  0  0]
 [ 0  0  0  0  

#### Метод опорных векторов с ялинейным ядром показал себя лучше всех

## 7. (1 балл) Обучите модель классификации Перцептрон. Рассчитайте метрики accuracy и выведите таблицу классификации.

In [22]:
model_mlp = MLPClassifier(random_state=35)

tm1 = time.time()
model_mlp.fit(X_train, y_train)
tm = time.time() - tm1

y_pred = model_mlp.predict(X_test)

ac = metrics.accuracy_score(y_test, y_pred)

modeles_comp.loc[modeles_comp.shape[0]] = ['mlp', tm, ac]

ac

0.3486238532110092

In [23]:
print(confusion_matrix(y_test, y_pred))

[[ 0  9  0  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  0  0]
 [ 0  7  7  0  0  0  0  0  0]
 [ 0  2  0  0  0  0 12  0  0]
 [ 0  0  0  0  0  0 12  0  0]
 [ 0  0  0  0  0  0 16  0  0]
 [ 0  0  0  0  0  0 14  0  0]
 [ 0  0  0  0  0  0 10  0  0]
 [ 0  0  0  0  0  0  3  0  7]]


## 8. (2 балла) Выведите итоговую таблицу сравнения всех моделей. В таблице должна быть информация о эффективности и времени обучения каждой модели. Сделайте вывод о применимости и эффективности моделей для классификации объектов в данной задаче.

In [24]:
modeles_comp

,Модель,Время,Точнось
0,linear,0.082401,1.000000
1,poly 2,0.122438,1.000000
2,poly 3,0.131543,0.972477
3,poly 4,0.141396,0.990826
4,poly 5,0.687149,0.990826
5,svc linear,0.007605,1.000000
6,svc poly,0.002665,0.706422
7,svc rbf,0.005006,0.568807
8,svc sigmoid,0.003811,0.091743
9,mlp,0.027200,0.348624


In [25]:
modeles_comp.sort_values(['Точнось', 'Время'], ascending=[False, True])

,Модель,Время,Точнось
5,svc linear,0.007605,1.000000
0,linear,0.082401,1.000000
1,poly 2,0.122438,1.000000
3,poly 4,0.141396,0.990826
4,poly 5,0.687149,0.990826
2,poly 3,0.131543,0.972477
6,svc poly,0.002665,0.706422
7,svc rbf,0.005006,0.568807
9,mlp,0.027200,0.348624
8,svc sigmoid,0.003811,0.091743


#### Вывод: Линейная модель довольно хорошо показала себя судя по точности, а так же и по времени т. к. она является самой примитивной моделью, что позовляет ей обучаться быстрее всех. Но победителем вышел Метод Опорных Векторов с линейным ядром, т. к. время обучения у него куда меньше. Остальные модели не особо хорошо показали себя на моих данных.